In [41]:
import pickle
from datetime import timedelta, datetime
from tqdm import tqdm
import json
from collections import defaultdict, Counter
from sklearn.linear_model import LogisticRegression
from scipy import sparse
import numpy as np
import requests
import pandas as pd

## 1. Прочитайте и проанализируйте данные

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:

+ взять в тренировочный набор турниры с dateStart из 2019 года
+ в тестовый — турниры с dateStart из 2020 года

In [8]:
with open("data/tournaments.pkl", "rb") as f:
    tournaments = pickle.load(f)
    
with open("data/results.pkl", "rb") as f:
    results = pickle.load(f)    

In [13]:
date_train_start = datetime.strptime('2019-01-01', '%Y-%m-%d')
date_val_start = datetime.strptime('2020-01-01', '%Y-%m-%d')

tournaments_train = []
tournaments_val = []

max_player_id = 0
max_team_id = 0

for key in tqdm(tournaments.keys(), position=0, leave=False):
    tournament_date = datetime.fromisoformat(tournaments[key]['dateStart']).replace(tzinfo=None)
    if tournament_date >= date_train_start:
        
        # Если нету команд, то пропускаем
        if not results[key]:
            continue
        
        total_questions = set()
        for team in results[key]:
            if team.get('mask') is not None:
                total_questions.add(len(team['mask']))
        
        # Пропустим турниры, где разное число вопросов
        if len(total_questions) > 1:
            continue
            
        tournament = {}
        tournament['id'] = tournaments[key]['id']
        tournament['teams'] = []
        
        for team in results[key]:
            # Уберем команды, где нет ответов или в ответах есть 'X', '?'
            if team.get('mask') is None or team.get('mask').replace('1', '').replace('0', ''):
                continue
                
            if team['team']['id'] > max_team_id:
                max_team_id = team['team']['id']
            
            team_dict = dict()
            team_dict['id'] = team['team']['id']
            team_dict['mask'] = team['mask']
            team_dict['members'] = []
            
            for member in team['teamMembers']:
                
                player_id = member['player']['id']
                if player_id > max_player_id:
                    max_player_id = player_id
                    
                team_dict['members'].append(player_id)
                
            tournament['teams'].append(team_dict)
        
        if not tournament['teams']:
            continue
        
        # Сразу будем делить на трейн и валидацию
        if tournament_date < date_val_start:
            tournaments_train.append(tournament)
        else:
            tournaments_val.append(tournament)

In [17]:
# Сохраним нужное, чтобы потом быстро подгружать
with open('data/tournaments_train.json', 'w') as file:
    json.dump(tournaments_train, file)
    
with open('data/tournaments_val.json', 'w') as file:
    json.dump(tournaments_val, file)

In [19]:
with open('data/tournaments_train.json', 'r') as file:
    tournaments_train = json.load(file)
    
with open('data/tournaments_val.json', 'r') as file:
    tournaments_val = json.load(file)
    
with open('data/players.pkl', 'rb') as file:
    players = pickle.load(file)

## 2. Baseline-модель

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:

+ повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации
+ в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса
+ для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков

Построим tf - idf для каждого игрока

In [23]:
members = set()
member_answers = defaultdict(int)
member_right_answers = defaultdict(int)
member_tours = defaultdict(int)

for tournament in tournaments_train:
    tournament_answers = []
    for team in tournament['teams']:
        for member in team['members']:
            member_answers[member] += len(team['mask'])
            member_right_answers[member] = sum(list(map(int, team['mask'])))
            member_tours[member] += 1
            members.add(member)
        

member_to_idx = {member:idx for idx, member in enumerate(members)}
idx_to_members = {idx:member for member, idx in member_to_idx.items()}
member_1000 = [member for member, answers in member_answers.items() if answers > 1000]

In [34]:
member_idxs = []
question_idxs = []
team_ids = []
tournament_ids = []
results = []
member_questions_count = []

questions_count = 0
for tournament in tqdm(tournaments_train, position=0, leave=False):
    tour_questions_count = len(tournament['teams'][0]['mask'])
    for team in tournament['teams']:
        team_answers = list(map(int, team['mask']))
        for q in range(tour_questions_count):
            for member in team['members']:
                member_idxs.append(member_to_idx[member])
                question_idxs.append(len(member_to_idx) + questions_count + q)
                team_ids.append(team['id'])
                tournament_ids.append(tournament['id'])
                results.append(team_answers[q])
                member_questions_count.append(member_answers[member])
    questions_count += tour_questions_count
    
    
X = sparse.lil_matrix((len(member_idxs), len(member_to_idx) + questions_count),  dtype=int)
X[range(len(member_idxs)), member_idxs] = 1
X[range(len(member_idxs)), question_idxs] = 1
y = np.array(results)

dim0, dim1 = X.shape

In [35]:
lr = LogisticRegression()

In [36]:
lr.fit(X, y)

D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

В качестве рейтинга возьмем коэфициенты логистической регрессии

In [38]:
question_rating = lr.coef_[0][len(member_to_idx):]
rating = lr.coef_[0][:len(member_to_idx)]
rating_list = []
for idx, member in idx_to_members.items():
    item = {
        'score': rating[idx],
        'id': member,
        'name': players[member]['name'] + ' ' + players[member]['surname'],
        'questions_count': member_answers[member],
    }
    rating_list.append(item)
    
sorted_rating = sorted(rating_list, key=lambda x: x['score'], reverse=True)

In [48]:
def get_member_position(id):
    url = f'https://rating.chgk.info/api/players/{id}/rating/last'
    position = -1
    try:
        position = requests.get(url).json()['rating_position']
        position = int(position)
    except:
        pass
    
    return position

In [49]:
df_rating = pd.DataFrame(sorted_rating)[:50]
df_rating['actual_position'] = df_rating['id'].apply(get_member_position)
df_rating['questions_count'] = df_rating['id'].apply(lambda x: member_answers[x])
df_rating.head(25)

,score,id,name,questions_count,actual_position
0,3.811086,27403,Максим Руссо,1796,5
1,3.559784,4270,Александра Брутер,2240,6
2,3.430787,37047,Мария Юнгер,452,589
3,3.398291,30152,Артём Сорожкин,4006,1
4,3.316383,20691,Станислав Мереминский,1370,38
5,3.289878,28751,Иван Семушин,3071,3
6,3.286252,34328,Михаил Царёв,366,310
7,3.280127,27822,Михаил Савченков,2666,2
8,3.277498,3843,Светлана Бомешко,336,3926
9,3.239502,18036,Михаил Левандовский,1113,8


In [50]:
top_50_count = df_rating[df_rating['actual_position'] <= 100].shape[0]

print(f'В топ-50 рейтинга модели попали {top_50_count} игроков из топ-100 реального рейтинга')

В топ-50 рейтинга модели попали 27 игроков из топ-100 реального рейтинга


Уберем игроков, которые сыграли меньше 1000 вопросов.

In [47]:
df_rating_1000 = pd.DataFrame(list(filter(lambda x: x['id'] in member_1000, sorted_rating)))[:50]
df_rating_1000['actual_position'] = df_rating_1000['id'].apply(get_member_position)
df_rating_1000['questions_count'] = df_rating_1000['id'].apply(lambda x: member_answers[x])
df_rating_1000.head(25)

,score,id,name,questions_count,actual_position
0,3.811086,27403,Максим Руссо,1796,5
1,3.559784,4270,Александра Брутер,2240,6
2,3.398291,30152,Артём Сорожкин,4006,1
3,3.316383,20691,Станислав Мереминский,1370,38
4,3.289878,28751,Иван Семушин,3071,3
5,3.280127,27822,Михаил Савченков,2666,2
6,3.239502,18036,Михаил Левандовский,1113,8
7,3.222995,56647,Наталья Горелова,1769,349
8,3.200083,22935,Илья Новиков,1266,132
9,3.187588,30270,Сергей Спешков,3017,4


In [45]:
top_50_count = df_rating_1000[df_rating_1000['actual_position'] <= 100].shape[0]

print(f'В топ-50 рейтинга модели попали {top_50_count} игроков из топ-100 реального рейтинга')

В топ-50 рейтинга модели попали 39 игроков из топ-100 реального рейтинга
